In [120]:
import dgl
import numpy as np
import torch
import networkx as nx
import os

from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from dgl.nn import NNConv, EdgeConv
from dgl.nn.pytorch import Sequential as dglSequential
from dgl.dataloading import GraphDataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm

In [11]:
def build_graph(jet):
    """return a fully connected graph: 
        - n nodes -> n particles
        - n**2 edges -> each p is connected to all the others"""
    g = dgl.graph()
    n = len(jet)
    g.add_nodes(n)
    
    src = []; dst = []
    for i in range(n):
        for j in range(n):
            src.append(i)
            dst.append(j)

    graph.add_edges(src, dst)

    node_feat = jet # should be a tensor of shape [n, 5] where 5 are the features of each particle
    edge_feat = jet*jet # should be tensor of shape [n**2, 6 ]
    # the first n rows refer to the edges between the first particle and the other n ones

    graph.ndata['feat'] = node_feat
    graph.edata['edge_feat'] = edge_feat

    return g



## Dataloader

In [4]:
data_path = '../data/graphdataset/'

['graphs0-20000.dgl',
 'graphs20000-40000.dgl',
 'graphs40000-60000.dgl',
 'graphs60000-80000.dgl',
 'graphs80000-100000.dgl']

In [20]:
class GraphDataset(Dataset):
    def __init__(self, data_path, n_files=None, transform=None):

        self.fnames    = [fname for fname in os.listdir(data_path) if fname.endswith('.dgl')]
        self.data_path = data_path

        if n_files:
            self.fnames = self.fnames[:n_files]

        self.transform = transform     

        self.load_graphs()


    def load_graphs(self):
        self.graphs = []

        for fname in tqdm(self.fnames, 'Reading files'):
            graphs = dgl.load_graphs(os.path.join(self.data_path, fname))[0]
            self.graphs.extend(graphs)


    def __len__(self):

        return len(self.graphs)
    

    def __getitem__(self, idx):

        x = self.graphs[idx]
        
        if self.transform:
            x = self.transform(x)
        
        return x

In [154]:
dataset = GraphDataset(data_path, 2)
train_dataloader = GraphDataLoader(dataset, batch_size=1)

Reading files:   0%|          | 0/2 [00:00<?, ?it/s]

In [157]:
for batch in train_dataloader:
    break
batch

Graph(num_nodes=72, num_edges=5184,
      ndata_schemes={'f': Scheme(shape=(7,), dtype=torch.float32)}
      edata_schemes={'d': Scheme(shape=(3,), dtype=torch.float32)})

In [164]:
dim = int(np.sqrt(batch.edata['d'].shape[0]))
batch.edata['d'].reshape((dim, dim, 3))[1,1,:]

tensor([1., 1., 1.])

## Simulated Graph
50 particelle, 7 features per particle, each node is connected to all the others so n**2 edges, each edge has 3 different features

In [43]:
n = 50 
src = []; dst = []
for i in range(n):
    for j in range(n):
        src.append(i)
        dst.append(j)

g = dgl.graph((src, dst))

nodes = torch.randn((n, 7))
edges = torch.randn((n**2, 3))

g.ndata['feat'] = nodes
g.edata['edge_feat'] = edges


In [72]:
def MLP(efeat, ch=[256,128,64,32]):

    mlp = nn.Sequential(
        nn.Linear(3, ch[0]),
        nn.Dropout(0.2),
        nn.ReLU(),
        
        nn.Linear(ch[0],ch[1]),
        nn.Dropout(0.2),
        nn.ReLU(),
        
        nn.Linear(ch[1],ch[2]),
        nn.Dropout(0.2),
        nn.ReLU(),

        nn.Linear(ch[2], ch[3]),
        nn.Dropout(0.2),
        nn.ReLU(),

        nn.Linear(ch[3], 7*128),
        nn.Sigmoid()
    )
    
    return mlp(efeat)

In [74]:
conv = NNConv(7, 128, MLP)
res = conv(g, nodes, edges)
res.shape

torch.Size([50, 128])

## Encoder

In [108]:
class Encoder(nn.Module):
    
    def __init__(self, latent_space_dim, ch=[256,128,64,32]):
        super().__init__()

        self.mlp = nn.Sequential(
            nn.Linear(3, ch[0]),
            nn.Dropout(0.2),
            nn.ReLU(),
            
            nn.Linear(ch[0],ch[1]),
            nn.Dropout(0.2),
            nn.ReLU(),
            
            nn.Linear(ch[1],ch[2]),
            nn.Dropout(0.2),
            nn.ReLU(),

            nn.Linear(ch[2], ch[3]),
            nn.Dropout(0.2),
            nn.ReLU(),

            nn.Linear(ch[3], 7*128),
            nn.Sigmoid()
            )
        
        self.conv = dglSequential(
            NNConv(
                in_feats  = 7,  # number of node features
                out_feats = 128, # output number of node features
                edge_func = self.mlp),
            EdgeConv(128, 64),
            EdgeConv(64, 32),
            EdgeConv(32, latent_space_dim)
        )

    def forward(self, graph, n_feat=None):

        x = self.conv(graph, n_feat if n_feat else graph.ndata['f'], graph.edata['d'])
        return x
    

In [109]:
encoder = Encoder(4)
encoded = encoder(batch)
encoded.shape

torch.Size([101, 4])

In [110]:
batch.ndata['l'] = encoded

## Decoder

In [111]:
class Decoder(nn.Module):
    
    def __init__(self, latent_space_dim):
        super().__init__()
        
        self.node_reconstruct = dglSequential(
            EdgeConv(latent_space_dim, 32),
            EdgeConv(32, 64),
            EdgeConv(64, 128),
            EdgeConv(128, 7))# output are the reconstructed node features

        self.edge_reconstruct = dglSequential(
            EdgeConv(latent_space_dim, 32),
            EdgeConv(32, 64),
            EdgeConv(64, 128),
            EdgeConv(128, 32),
            EdgeConv(32,16),
            EdgeConv(16,8)
        )

    def forward(self, graph, n_feat=None):
        
        if n_feat is None:
            n_feat = graph.ndata['l']

        # node reconstruction
        n = self.node_reconstruct(graph, n_feat)
        
        # edges reconstruction
        e1 = self.edge_reconstruct(graph, n_feat)
        e2 = self.edge_reconstruct(graph, n_feat)
        e3 = self.edge_reconstruct(graph, n_feat)

        # inner product o matmul?
        e1 = torch.inner(e1, e1)  #their elements are A_{ij}
        e2 = torch.inner(e2, e2) 
        e3 = torch.inner(e3, e3)

        return n, e1, e2, e3
    

In [112]:
decoder = Decoder(4)
decoded = decoder(batch)

In [114]:
autoencoder = dglSequential(encoder, decoder)
n, e1, e2, e3 = autoencoder(batch)

## Training

In [116]:
### Initialize the two networks
encoded_space_dim = 6
encoder = Encoder(latent_space_dim=encoded_space_dim)
decoder = Decoder(latent_space_dim=encoded_space_dim)

### Learning Rate and Loss customize 

In [118]:
def custom_loss(output, target):
    """ pred and real are two tuples which contains:
    real: real nodes and edges
    real: real nodes and edges"""

    # regularization parameters    
    l_node = 0.3
    l_edge = 1
    
    loss_fn = torch.nn.MSELoss()
    mse_losses = [torch.sqrt(loss_fn(pred, real)) for pred, real in zip(output, target)]

    return l_node * mse_losses[0] + l_edge * torch.sum(torch.tensor(mse_losses[1:]))


In [121]:
### Define an optimizer (both for the encoder and the decoder!)
lr = 1e-3 # Learning rate
loss_fn = custom_loss
params_to_optimize = [
    {'params': encoder.parameters()},
    {'params': decoder.parameters()}
]
optimizer = torch.optim.Adam(params_to_optimize, lr=lr)

# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

"""
usando questo bisogna fare scheduler.step(val_loss) alla fine di un epoca
"""
lr_scheduler = ReduceLROnPlateau(
    optimizer,
    mode = 'min',
    factor=0.5,
    patience=5,
    cooldown=5,
    min_lr=10e-8 # this should be the minimum value of the lr at which we stop the training
    )

Selected device: cuda


### training and validate function

In [171]:
### Training function
def train_epoch(encoder, decoder, device, dataloader, loss_fn, optimizer):
    # Set train mode for both the encoder and the decoder
    encoder.train()
    decoder.train()

    # abbiamo già definito l'optimizer nella cella precedente
    
    losses = []
    pbar = tqdm(dataloader, 'Steps')
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for batch in pbar: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        
        batch = batch.to(device)
        #encode
        y_encoder_pred = encoder(batch)
        #update graph node features with their latent space representation
        batch.ndata['l'] = y_encoder_pred
        #decode
        y_decoder_pred = decoder(batch)

        dim = int(np.sqrt(batch.edata['d'].shape[0]))
        loss = loss_fn(y_decoder_pred, [batch.ndata['f']] + [batch.edata['d'][:,i].reshape((dim, dim)) for i in range(3)])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.detach().cpu().numpy())
        pbar.set_postfix_str(f'loss: {losses[-1]:.2f}')
        
    losses = np.mean(losses)
    return losses

In [ ]:
### Testing function
def test_epoch(encoder, decoder, device, dataloader, loss_fn):
    # Set evaluation mode for encoder and decoder
    encoder.eval()
    decoder.eval()

    with torch.no_grad(): # No need to track the gradients
        # Define the lists to store the outputs for each batch
        conc_out = []
        conc_label = []
        for image_batch, _ in dataloader:
            # Move tensor to the proper device
            image_batch = image_batch.to(device)
            # Encode data
            encoded_data = encoder(image_batch)
            # Decode data
            decoded_data = decoder(encoded_data)
            # Append the network output and the original image to the lists
            conc_out.append(decoded_data.cpu())
            conc_label.append(image_batch.cpu())
        # Create a single tensor with all the values in the lists
        conc_out = torch.cat(conc_out)
        conc_label = torch.cat(conc_label) 
        # Evaluate global loss
        val_loss = loss_fn(conc_out, conc_label)
    return val_loss.data

In [172]:
# train the model
epoch = 0
while(True):
    # train for one epoch
    print('EPOCH %d' % (epoch + 1))

    train_loss = train_epoch(
        encoder=encoder, 
        decoder=decoder, 
        device=device, 
        dataloader=train_dataloader, 
        loss_fn=loss_fn, 
        optimizer=optimizer)
    print(f'TRAIN - EPOCH {epoch+1} - loss: {train_loss}')

    '''# evaluate the model
    val_loss = test_epoch(
        encoder=encoder, 
        decoder=decoder, 
        device=device, 
        dataloader=test_dataloader, 
        loss_fn=loss_fn)
    # Print Validationloss
    print(f'VALIDATION - EPOCH {epoch+1} - loss: {val_loss}\n')'''

    # schedule the learning rate
    epoch +=1
    lr_scheduler.step(train_loss)

    if optimizer.param_groups[0]['lr'] < 1e-8:
        break

EPOCH 1


Steps:   0%|          | 0/40000 [00:00<?, ?it/s]